# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [ ]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [ ]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [ ]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [ ]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [ ]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

3. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations: Knebworth House, Knebworth, Hertfordshire, England, UK (Wayne Manor; exterior) Helpful•21 0 
Acton Lane Power Station, Acton Lane, Acton, ...

4. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda ...

5. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Here are 12 real-world Batman movie locations that you can visit now · St. George's Hall, Liverpool · Two Temple 
Place, London · Wollaton Hall, ...

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOorSaHBbLAYYKeV8swZKGVHjNXn9tcENa1rjyXx
XMusxnXDgWT34)
Source: LA Today

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

7. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

8. [The Batman (2022) Locations](https://www.latlong.net/location/the-batman-2022-locations-427)
Source: Latitude and Longitude Finder

The Batman (2022) was filmed in Liverpool and London, England, UK. Filming also took place at Cardington Studios 
and Warner Bros. Leavesden Studios.

9. [Where was The Dark Knight filmed?](https://giggster.com/guide/movie-location/where-was-the-dark-knight-filmed)
Source: Giggster

The Dark Knight locations were spread across the world with filming in Chicago in the United States, London in 
England, and Hong Kong in China.

Out: None

[Step 0: Duration 10.92 seconds| Input tokens: 2,399 | Output tokens: 160]

...

[Step 1: Duration 15.03 seconds| Input tokens: 5,858 | Output tokens: 1,279]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Extract locations and their coordinates for supercar factories                                                 
  supercar_locations = [                                                                                           
      ("Maranello, Italy (Ferrari)", 44.5403, 10.9921),                                                            
      ("Sant'Agata Bolognese, Italy (Lamborghini)", 44.5208, 11.3264),                                             
      ("San Cesario sul Panaro, Italy (Pagani)", 44.6423, 10.9917),                                                
      ("Bologna, Italy (Ducati)", 44.4937, 11.3426),                                                               
      ("Woking, UK (McLaren)", 51.3774, -0.5714),                                                                  
      ("San Diego, CA, USA (Koenigsegg)", 32.7157, -117.1611),                                                     
      ("Amersfoort, Netherlands (Aston Martin)", 52.0118, 5.5975),                                                 
      ("Los Angeles, CA, USA (Hennessey Performance Engineering)", 34.0522, -118.2437),                            
      ("Shanghai, China (Shanghai Auto Show)", 31.2304, 121.4737),                                                 
      ("Dubai, UAE (Dubai Autodrome)", 25.2769, 55.2962)                                                           
  ]                                                                                                                
                                                                                                                   
  # Calculate travel time for each supercar factory                                                                
  supercar_travel_times = []                                                                                       
  for location, lat, lon in supercar_locations:                                                                    
      travel_time = calculate_cargo_travel_time(origin_coords=(lat, lon), destination_coords=gotham_coords)        
      supercar_travel_times.append((location, travel_time))                                                        
                                                                                                                   
  # Create DataFrame for supercar factories                                                                        
  supercar_df = pd.DataFrame(supercar_travel_times, columns=['Location', 'Travel Time to Gotham (hours)'])         
                                                                                                                   
  # Filter supercar factories to match the travel times of Batman filming locations                                
  batman_travel_times_set = set(batman_df['Travel Time to Gotham (hours)'])                                        
  filtered_supercar_df = supercar_df[supercar_df['Travel Time to Gotham (hours)'].isin(batman_travel_times_set)]   
                                                                                                                   
  # Combine Batman filming locations and filtered supercar factories into a single DataFrame                       
  combined_df = pd.concat([batman_df, filtered_supercar_df], ignore_index=True)                                    
                                                                                                                   
  print(combined_df)                                                                                               
                                                                                                                   
  final_answer(combined_df)                                                                                        
 ────────────────────────────────────────────────────────

Execution logs:
                                             Location  \
0          Necropolis Cemetery, Glasgow, Scotland, UK   
1           St. George's Hall, Liverpool, England, UK   
2               Two Temple Place, London, England, UK   
3              Wollaton Hall, Nottingham, England, UK   
4   Knebworth House, Knebworth, Hertfordshire, Eng...   
5   Acton Lane Power Station, Acton Lane, Acton, E...   
6               Queensboro Bridge, New York City, USA   
7                     Wall Street, New York City, USA   
8          Mehrangarh Fort, Jodhpur, Rajasthan, India   
9                         Turda Gorge, Turda, Romania   
10                                       Chicago, USA   
11                                   Hong Kong, China   
12  Cardington Studios, Northamptonshire, England, UK   
13  Warner Bros. Leavesden Studios, Hertfordshire,...   
14                     Westwood, Los Angeles, CA, USA   
15                               Woking, UK (McLaren)   

    Travel Time to Gotham (hours)  
0                            8.60  
1                            8.81  
2                            9.17  
3                            9.00  
4                            9.15  
5                            9.16  
6                            1.01  
7                            1.00  
8                           18.34  
9                           11.89  
10                           2.68  
11                          19.99  
12                           9.10  
13                           9.13  
14                           6.79  
15                           9.13  

Out - Final answer:                                              Location  \
0          Necropolis Cemetery, Glasgow, Scotland, UK   
1           St. George's Hall, Liverpool, England, UK   
2               Two Temple Place, London, England, UK   
3              Wollaton Hall, Nottingham, England, UK   
4   Knebworth House, Knebworth, Hertfordshire, Eng...   
5   Acton Lane Power Station, Acton Lane, Acton, E...   
6               Queensboro Bridge, New York City, USA   
7                     Wall Street, New York City, USA   
8          Mehrangarh Fort, Jodhpur, Rajasthan, India   
9                         Turda Gorge, Turda, Romania   
10                                       Chicago, USA   
11                                   Hong Kong, China   
12  Cardington Studios, Northamptonshire, England, UK   
13  Warner Bros. Leavesden Studios, Hertfordshire,...   
14                     Westwood, Los Angeles, CA, USA   
15                               Woking, UK (McLaren)   

    Travel Time to Gotham (hours)  
0                            8.60  
1                            8.81  
2                            9.17  
3                            9.00  
4                            9.15  
5                            9.16  
6                            1.01  
7                            1.00  
8                           18.34  
9                           11.89  
10                           2.68  
11                          19.99  
12                           9.10  
13                           9.13  
14                           6.79  
15                           9.13  

[Step 2: Duration 14.67 seconds| Input tokens: 12,077 | Output tokens: 2,438]

In [ ]:
result

,Location,Travel Time to Gotham (hours)
0,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
1,"St. George's Hall, Liverpool, England, UK",8.81
2,"Two Temple Place, London, England, UK",9.17
3,"Wollaton Hall, Nottingham, England, UK",9.00
4,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.15
5,"Acton Lane Power Station, Acton Lane, Acton, E...",9.16
6,"Queensboro Bridge, New York City, USA",1.01
7,"Wall Street, New York City, USA",1.00
8,"Mehrangarh Fort, Jodhpur, Rajasthan, India",18.34
9,"Turda Gorge, Turda, Romania",11.89


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location, visit the source URL to confirm the coordinates.
3. Calculate the cargo plane transfer time from each filming location to Gotham.
4. Perform a web search to find a comprehensive list of supercar factories worldwide.
5. For each supercar factory, visit the source URL to confirm the coordinates.
6. Calculate the cargo plane transfer time from each supercar factory to Gotham.
7. Filter both lists to include only locations with the same cargo plane transfer time.
8. Organize the filtered locations and their details into a pandas dataframe.
9. Provide the final answer as the pandas dataframe.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  Thought: I will perform a web search to find a comprehensive list of Batman filming locations worldwide.         
  Code:                                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                                             Location  \
0          Necropolis Cemetery, Glasgow, Scotland, UK   
1           St. George's Hall, Liverpool, England, UK   
2               Two Temple Place, London, England, UK   
3              Wollaton Hall, Nottingham, England, UK   
4   Knebworth House, Knebworth, Hertfordshire, Eng...   
5   Acton Lane Power Station, Acton Lane, Acton, E...   
6               Queensboro Bridge, New York City, USA   
7                     Wall Street, New York City, USA   
8          Mehrangarh Fort, Jodhpur, Rajasthan, India   
9                         Turda Gorge, Turda, Romania   
10                                       Chicago, USA   
11                                   Hong Kong, China   
12  Cardington Studios, Northamptonshire, England, UK   
13  Warner Bros. Leavesden Studios, Hertfordshire,...   
14                     Westwood, Los Angeles, CA, USA   
15                               Woking, UK (McLaren)   

    Travel Time to Gotham (hours)  
0                            8.60  
1                            8.81  
2                            9.17  
3                            9.00  
4                            9.15  
5                            9.16  
6                            1.01  
7                            1.00  
8                           18.34  
9                           11.89  
10                           2.68  
11                          19.99  
12                           9.10  
13                           9.13  
14                           6.79  
15                           9.13

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Manually specify coordinates for Batman filming locations                                                      
  batman_coordinates = {                                                                                           
      "St George's Hall, Liverpool, England, UK": (53.4084, -2.9916),                                              
      "Cardington Airship Hangar 1, Northamptonshire, England, UK": (52.2357, -0.4343),                            
      "Bridge of Sighs, Glasgow Necropolis, Glasgow, Scotland, UK": (55.8642, -4.2894),                            
      "Cherry Lane entrance, Anfield Cemetery, Liverpool, England, UK": (53.4084, -2.9916),                        
      "Wishart Street, Glasgow, Scotland, UK": (55.8642, -4.2894),                                                 
      "Chicago, USA": (41.8781, -87.6298),                                                                         
      "London, England, UK": (51.5074, -0.1278),                                                                   
      "Bedfordshire, England, UK": (52.1307, -0.4140),                                                             
      "Hong Kong, China": (22.3964, 114.1095)                                                                      
  }                                                                                                                
                                                                                                                   
  # Manually specify coordinates for supercar factories                                                            
  supercar_factories = {                                                                                           
      "Lamborghini, Sant'Agata Bolognese, Italy": (44.5403, 11.3467),                                              
      "Ferrari, Maranello, Italy": (44.5646, 10.8941),                                                             
      "Porsche, Leipzig, Germany": (51.3397, 12.3731),                                                             
      "McLaren, Woking, UK": (51.3774, -0.5713),                                                                   
      "Aston Martin, Gaydon, UK": (52.3705, -1.3104),                                                              
      "Koenigsegg, Ängelholm, Sweden": (56.1589, 12.8906),                                                         
      "Pagani, San Cesario sul Panaro, Italy": (44.7561, 10.9868),                                                 
      "Bugatti, Molsheim, France": (48.6922, 7.7486),                                                              
      "Rimac, Zagreb, Croatia": (45.8150, 15.9819),                                                                
      "Hypercar, Abu Dhabi, UAE": (24.4667, 54.3667)                                                               
  }                                                                                                                
                                                                                                                   
  # Gotham's coordinates                                                                                           
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Function to calculate cargo travel time                                                                        
  def calculate_travel_time(origin_coords, destination_co

Execution logs:
                                            Location  Travel Time (hours)
0  Bridge of Sighs, Glasgow Necropolis, Glasgow, ...                  8.6
1              Wishart Street, Glasgow, Scotland, UK                  8.6

Out - Final answer:                                             Location  Travel Time (hours)
0  Bridge of Sighs, Glasgow Necropolis, Glasgow, ...                  8.6
1              Wishart Street, Glasgow, Scotland, UK                  8.6

[Step 6: Duration 31.47 seconds| Input tokens: 83,622 | Output tokens: 5,145]

                                            Location  Travel Time (hours)
0  Bridge of Sighs, Glasgow Necropolis, Glasgow, ...                  8.6
1              Wishart Street, Glasgow, Scotland, UK                  8.6


In [ ]:
detailed_report

,Location,Travel Time (hours)
0,"Bridge of Sighs, Glasgow Necropolis, Glasgow, ...",8.6
1,"Wishart Street, Glasgow, Scotland, UK",8.6


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [ ]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                <think>
Okay, let's break down the problem step by step. The user wants a map showing Batman filming locations and supercar
factories with their cargo plane transfer times to Gotham. The total points should be at least six, with supercar 
factories having the same transfer times as some Batman locations.

First, I need to gather the Batman filming locations. Since I don't have that data, I'll need to use the web_agent 
to search for reliable sources. Once I have the list of locations with their coordinates, I can calculate the 
travel time from each to Gotham using the calculate_cargo_travel_time tool. 

Next, I need to find supercar factories. Again, the web_agent can help find their locations. For each factory, I'll
calculate the travel time to Gotham. Then, I have to match these times with the Batman locations' times. If a 
factory's time matches any of the Batman times, it gets included. If there aren't enough matches, I might need to 
adjust by finding more factories or relaxing the time matching criteria, but the user specified "same time," so 
I'll stick to exact matches as much as possible.

Once I have at least six points (combination of Batman locations and matching factories), I'll compile them into a 
dataset. The example code uses a DataFrame with lat, lon, name, and color based on travel time. I'll structure my 
data similarly. Finally, I'll plot the points on a map using plotly, color-coding by travel time, and save the 
image.
</think>

1. Task the web agent to compile a list of all Batman movie filming locations worldwide, including their geographic
coordinates (latitude/longitude).  
2. For each Batman filming location, use the `calculate_cargo_travel_time` tool to determine the cargo plane 
transfer time to Gotham (40.7128° N, 74.0060° W).  
3. Task the web agent to identify factories of supercar manufacturers globally, including their geographic 
coordinates.  
4. For each supercar factory, calculate its cargo plane transfer time to Gotham using the same tool.  
5. Filter the supercar factories to retain only those with transfer times matching the Batman locations’ calculated
times. Combine these with Batman locations to ensure at least 6 total points.  
6. Create a DataFrame with columns for latitude, longitude, location/factory name, and transfer time.  
7. Plot the combined data on a global map using `plotly.express`, coloring points by transfer time. Save the map to
`saved_map.png`.  
8. Return the saved map as the final answer.  


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
 [STEP 1]:                                                                                                         
Thought: I need to get the list of Batman filming locations with their coordinates. Since I don't have this data,  
I'll task the web_agent to search for it.                                                                          
Code:                                                                                                              
```py                                                                                                              
batman_locations_task = "Find all Batman movie filming locations worldwide. For each location, provide the name and
geographic coordinates (latitude, longitude). Use reliable sources like IMDb or Movie-Locations.com."              
web_agent_response = web_agent(task=batman_locations_task)                                                         
print(web_agent_response)                                                                                          
```<end_code>                                                                                                      
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations_task = "Find all Batman movie filming locations worldwide. For each location, provide the name  
  and geographic coordinates (latitude, longitude). Use reliable sources like IMDb or Movie-Locations.com."        
  web_agent_response = web_agent(task=batman_locations_task)                                                       
  print(web_agent_response)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Reconstruct batman_times with original structure (from Step 2)                                                 
  batman_data = [                                                                                                  
      ("Stearns Wharf, Santa Barbara", 34.4188, -119.6951, 6.93),                                                  
      ("Cliffs, Marineland", 33.9, -118.4, 6.80),                                                                  
      ("Bronson Caves, Griffith Park", 34.1347, -118.3215, 6.78),                                                  
      ("Alcatraz Island", 37.8267, -122.4233, 7.05),                                                               
      ("Knebworth House, UK", 51.8055, -0.4035, 9.13),                                                             
      ("Skaftafell, Iceland", 64.2128, -19.0649, 7.37),                                                            
      ("Chicago Board of Trade", 41.8879, -87.6235, 2.68),                                                         
      ("St Pancras Hotel, London", 51.5343, -0.1341, 9.17),                                                        
      ("Battersea Power Station", 51.4719, -0.1167, 9.17),                                                         
      ("Necropolis Cemetery", 55.8642, -4.2518, 8.60)                                                              
  ]                                                                                                                
                                                                                                                   
  supercar_data = [                                                                                                
      ("Ferrari Maranello", 44.58, 10.89, 10.72),                                                                  
      ("Lamborghini Sant'Agata", 44.54, 11.18, 10.75),                                                             
      ("Bugatti Molsheim", 48.69, 7.71, 10.10)                                                                     
  ]                                                                                                                
                                                                                                                   
  # Combine into DataFrame                                                                                         
  import pandas as pd                                                                                              
  df = pd.DataFrame(                                                                                               
      [                                                                                                            
          {                                                                                                        
              'name': name,                                                                                        
              'lat': lat,                                                                                          
              'lon': lon,                                                                                          
              'time': time,                                                                                        
              'type': 'Batman Location'                                                                            
          } for (name, lat, lon, time) in batman_data                                                              
      ][38;2;248;248;

Out - Final answer: saved_map.png

Feedback:  **Reasons Why Yes:**

1. **Plotting Method**: The plot was made using `px.scatter_mapbox`, which is the correct method as per the task requirements.
2. **Location Representation**: The map shows both Batman filming locations and supercar factories, fulfilling the request for a spatial representation.
3. **Color Coding**: Locations are color-coded based on travel time, providing a clear visual differentiation.
4. **Number of Points**: The plot displays at least six points, meeting the minimum requirement set by the task.

**Reasons Why No:**

1. **Matching Times**: The task required supercar factories to match Batman locations' transfer times. This plot doesn't specifically confirm any matching times.
2. **Transfer Time Calculation**: There was a logical error in handling the transfer times that wasn't corrected in plotting. 
3. **Variable Errors**: The plot creation initially faced issues due to handling errors in data structures, affecting data integrity.

**Final Decision:

[Step 6: Duration 37.00 seconds| Input tokens: 34,990 | Output tokens: 2,560]

'saved_map.png'

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)